In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
import time
import os
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Data Loading 

In [2]:
def get_results(y_pred, y_test):
    number_flagged_right=np.sum(y_pred[y_test==1]==1)
    number_flagged_wrong=np.sum(y_test==1)-number_flagged_right
    number_Null_right=np.sum(y_pred[y_test==0]==0)
    number_Null_wrong=np.sum(y_test==0)-number_Null_right
    print(f'Flagged Right: {number_flagged_right}')
    print(f'Flagged Wrong: {number_flagged_wrong}')
    print(f'Null Right: {number_Null_right}')
    print(f'Null Wrong: {number_Null_wrong}')

In [3]:
load_data = torch.load('../data.pt')
X = load_data["X"]
y = load_data["y"]
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.1, random_state=0)

In [4]:
j=0
for train_index, test_index in sss.split(X, y):
    if j==0:
        citi_X_train, citi_X_test = X[train_index], X[test_index]
        citi_y_train, citi_y_test = y[train_index], y[test_index]
    elif j==1:
        jpm_X_train, jpm_X_test = X[train_index], X[test_index]
        jpm_y_train, jpm_y_test = y[train_index], y[test_index]
    elif j==2:
        boa_X_train, boa_X_test = X[train_index], X[test_index]
        boa_y_train, boa_y_test = y[train_index], y[test_index]
    j+=1

In [5]:
X_test=np.vstack([citi_X_test,jpm_X_test,boa_X_test])
y_test=np.hstack([citi_y_test,jpm_y_test,boa_y_test])

In [6]:
weights = (citi_y_train==0).sum()/(1.0 *  (citi_y_train==1).sum())
model = XGBClassifier(tree_method='gpu_exact',objective='binary:logistic', n_estimators=300, scale_pos_weight=weights,n_jobs=6)

In [7]:
model.fit(citi_X_train, citi_y_train,eval_metric=['auc'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=6, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=773.6696428571429,
       seed=None, silent=True, subsample=1, tree_method='gpu_exact')

In [8]:
y_pred = model.predict(X_test)
get_results(y_pred, y_test)

Flagged Right: 2461
Flagged Wrong: 2
Null Right: 1893312
Null Wrong: 13011


/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
model.save_model('citi.model')

In [10]:
weights = (jpm_y_train==0).sum()/(1.0 *  (jpm_y_train==1).sum())
model2 = XGBClassifier(tree_method='gpu_exact',objective='binary:logistic', scale_pos_weight=weights, n_estimators=300,n_jobs=6)
model2.fit(jpm_X_train, jpm_y_train, xgb_model='citi.model',eval_metric=['auc'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=6, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=773.6696428571429,
       seed=None, silent=True, subsample=1, tree_method='gpu_exact')

In [11]:
y_pred = model2.predict(X_test)
get_results(y_pred, y_test)

Flagged Right: 2459
Flagged Wrong: 4
Null Right: 1893415
Null Wrong: 12908


/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [12]:
model2.save_model('jpm.model')

In [13]:
weights = (boa_y_train==0).sum()/(1.0 *  (boa_y_train==1).sum())
model3 = XGBClassifier(tree_method='gpu_exact', objective='binary:logistic', n_estimators=300,scale_pos_weight=weights, n_jobs=6)
model3.fit(boa_X_train, boa_y_train, xgb_model='jpm.model',eval_metric=['auc'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=6, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=773.6696428571429,
       seed=None, silent=True, subsample=1, tree_method='gpu_exact')

In [14]:
y_pred = model3.predict(X_test)
get_results(y_pred, y_test)

Flagged Right: 2458
Flagged Wrong: 5
Null Right: 1893792
Null Wrong: 12531


/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
model3.feature_importances_

array([0.14386928, 0.13212146, 0.13532706, 0.18767431, 0.23919494,
       0.05697515, 0.10483781], dtype=float32)

In [16]:
len(y_test)

1908786